# 火炬内部之旅(第一部分)
> 原文地址：[A Tour of PyTorch Internals (Part I)](https://pytorch.org/2017/05/11/Internals.html) 
> 希望从翻译这篇文章开始能认真的去读一下pytorch源码

本文进度  
- [X] 简介  
- [X] 扩展 Python 解释器  
- [ ] The THPTensor Type  
- [ ] Generic Builds  
- [ ] PyTorch 封装

火炬的基本单位是张量。 这篇文章将作为我们如何在 PyTorch 中实现张量的概述，这样用户可以从 Python shell 中与之交互。 特别是，我们想要回答四个主要问题:
* Pytorch 是如何通过定义可以从 Python 代码中操作的 Tensor 类型来扩展 Python 解释器的？
* Pytorch 是如何封装实际定义 Tensor 的属性和方法的 c 库的？
* Pytorch 如何编写 Tensor 方法的代码？
* Pytorch 的构建系统如何将所有这些组件来编译和生成一个可行的应用程序的？

## 扩展 Python 解释器

PyTorch 定义了一个新的`torch`包。 在这篇文章中，我们将主要介绍 `._C` 模块。 这个模块被称为"扩展模块"——用 c 编写的 Python 模块。 这些模块允许我们定义新的内置 python 类型(例如 `Tensor`) ，和调用 c/c++ 函数。 

`._C` 模块在 `torch/csrc/module.cpp` 中定义。 函数 `init_C()` / `PyInit__C()`函数创建`._C`模块并酌情添加方法定义。 `._C`模块被传递给一系列以 `__init()` 开头的函数，这些函数负责向模块中添加更多的对象，或者注册新的类型等。 

> 译注：关于`init_C()` / `PyInit__C()`这两个函数，前者在py2中被使用，后者在py3中被使用。

部分 `__init()` 函数如下：

```c
ASSERT_TRUE(THPDoubleTensor_init(module));
ASSERT_TRUE(THPFloatTensor_init(module));
ASSERT_TRUE(THPHalfTensor_init(module));
ASSERT_TRUE(THPLongTensor_init(module));
ASSERT_TRUE(THPIntTensor_init(module));
ASSERT_TRUE(THPShortTensor_init(module));
ASSERT_TRUE(THPCharTensor_init(module));
ASSERT_TRUE(THPByteTensor_init(module));
```

这些函数将不同类型的 `Tensor` 对象添加到 `_C` 模块，以便它们可以在模块中使用。 让我们来了解一下这些方法是如何工作的。

## The THPTensor Type

与底层的 TH 和 THC 库非常相似，PyTorch 定义了一个"通用的" Tensor 基类(实际上它的名字叫 THPTensor),这个基类用于生成许多不同的子类。 在考虑这个子类如何定义之前，让我们首先考虑如何在  Python 中定义一个新的内置类型，以及我们如何创建通用的 THPTensor 类。

Python 解释器将所有 Python 对象视为 `PyObject *` 类型的变量，它是所有 Python 对象的"基础类型"。 每个 Python 类型都包含一个针对对象的引用计数，以及指向对象类型对象的指针。 类型对象决定类型有什么属性。 例如，它可能包含一个与类型相关联的方法列表，列表中的每一个元素会有一个对应的 c 函数的实现。 该对象还包含表示其状态所必需的任何字段。

定义一个新类型的方法如下：

* 创建一个结构体，用来定义新的对象包含什么属性
* 为该类型定义一个**类型对象**

结构体本身可能非常简单。 在Python中，所有的浮点类型实际上都是堆上的对象。 这个 Python 浮点结构的定义是:

```C
typedef struct {
    PyObject_HEAD
    double ob_fval;
} PyFloatObject;
```

`PyObject_HEAD` 是一个宏，它定义了了实现对象的引用计数的代码，以及一个指向对应类型对象的指针。 因此，在这种情况下，要实现浮点，唯一需要的其他"状态"是浮点值本身。

现在，让我们看看 THPTensor 类型的结构:

```c
struct THPTensor {
    PyObject_HEAD
    THTensor *cdata;
};
```

很简单，对吧？ 我们只是存储一个底层的 TH 张量指针，然后做一个封装。

关键部分是为新类型定义"类型对象"。 我们的 Python 浮点的类型对象的一个示例定义采用了以下形式:

```c
static PyTypeObject py_FloatType = {
    PyVarObject_HEAD_INIT(NULL, 0)
    "py.FloatObject",          /* tp_name */
    sizeof(PyFloatObject),     /* tp_basicsize */
    0,                         /* tp_itemsize */
    0,                         /* tp_dealloc */
    0,                         /* tp_print */
    0,                         /* tp_getattr */
    0,                         /* tp_setattr */
    0,                         /* tp_as_async */
    0,                         /* tp_repr */
    0,                         /* tp_as_number */
    0,                         /* tp_as_sequence */
    0,                         /* tp_as_mapping */
    0,                         /* tp_hash  */
    0,                         /* tp_call */
    0,                         /* tp_str */
    0,                         /* tp_getattro */
    0,                         /* tp_setattro */
    0,                         /* tp_as_buffer */
    Py_TPFLAGS_DEFAULT,        /* tp_flags */
    "A floating point number", /* tp_doc */
};
```

对类型对象的最简单的理解方式是，它定义了一组字段，这些字段将成为对象属性。 例如，`tp_basicsize` 字段的大小被设置为 `sizeof(PyFloatObject)`。 这是为了让 Python 为 `PyFloatObject` 调用 `PyObject_New()` 时(实例化一个 `PyFloatObject` 对象)知道要分配多少内存。 你可以设置的全部字段列表在 CPython 源码的 [`object.h`](https://github.com/python/cpython/blob/master/Include/object.h) 中定义。

我们需要为 `THPTensor` 定义的类型对象是 `THPTensorType` ，在 `csrc/generic/Tensor.cpp` 中被定义。这个对象为 `THPTensor` 定义了名称、大小、映射方法等。

举个例子，让我们来看看 PyTypeObject 中设置的 `tp_new` 函数:

```c
PyTypeObject THPTensorType = {
  PyVarObject_HEAD_INIT(NULL, 0)
  ...
  THPTensor_(pynew), /* tp_new */
};
```

这个 `tp_new` 函数使对象创建成为可能。 它负责创建(而不是初始化)该类型的对象，并且与 Python 中的 `__new__()` 相当。这个 C 实现是一个静态方法，它传递被实例化的类型和初始化需要的参数，并返回一个新被创建的对象。

```c
static PyObject * THPTensor_(pynew)(PyTypeObject *type, PyObject *args, PyObject *kwargs)
{
  HANDLE_TH_ERRORS
  Py_ssize_t num_args = args ? PyTuple_Size(args) : 0;

  THPTensorPtr self = (THPTensor *)type->tp_alloc(type, 0);
// more code below
```

The first thing our new function does is allocate the THPTensor. It then runs through a series of initializations based off of the args passed to the function. For example, when creating a THPTensor x from another THPTensor y, we set the newly created THPTensor’s cdata field to be the result of calling THTensor_(newWithTensor) with the y’s underlying TH Tensor as an argument. Similar constructors exist for sizes, storages, NumPy arrays, and sequences.

我们的新函数所做的第一件事就是分配为新的 `THPTensor` 分配内存。然后它通过运行一系列初始化函数来使用传入的参数。例如，当从另一个 `THPTensor y` 创建一个 `THPTensor x` 时，变量 x 的 cdata 字段的值，将会是调用 `THTensor_(y)` 的返回值(这个函数是 C 函数，其中的y指的是这个python对象所对应的底层THPTensor对象)。将新创建的 THPTensor 的 cdata 场设置为 THTensor (newWithTensor)作为参数。 类似的构造函数存在于大小、存储器、 NumPy 数组和序列上。

> 此处应该为最后一句话添加译注

** 注意：我们只单独使用 `tp_new` ，而不是组合使用 `tp_new` 和 `tp_init` (对应python中的 __init__()函数)。

在 `Tensor.cpp` 中定义的另一个重要的东西是索引的工作方式。 PyTorch 支持 Python 的映射协议。 这让我们可以做以下的事情:

```python
x = torch.Tensor(10).fill_(1)
y = x[3] // y == 1
x[4] = 2
// etc.
```

** 注意：此索引扩展到具有多个维度的 Tensor

通过定义这里描述的三种映射方法，我们能够使用[]样式符号。

The most important methods are THPTensor_(getValue) and THPTensor_(setValue) which describe how to index a Tensor, for returning a new Tensor/Scalar, or updating the values of an existing Tensor in place. Read through these implementations to better understand how PyTorch supports basic tensor indexing.

最重要的方法是 `THPTensor_(getValue)` 和 `THPTensor_(setValue)` ，它描述了如何索引张量，如何返回新的张量 / 标量，或原第更新现有张量的值。通读这些实现来更好地理解 PyTorch 如何支持基本的张量索引。

## Generic Builds

## PyTorch 封装